# <center>Data Mining Project 2 Spring semester 2018-2019</center>
## <center>Παναγιώτης Ευαγγελίου &emsp; 1115201500039</center>
## <center>Ευάγγελος Σπίθας &emsp;&emsp;&emsp;&ensp; 1115201500147</center>

___

## Do all the necessary imports for this notebook

In [1]:
import numpy as np
import pandas as pd
import calendar

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## __Read data__

In [2]:
initialDataFrame = pd.read_csv('../data/crime.csv', engine='python')

initialDataFrame # printToBeRemoved

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182080058,2403,Disorderly Conduct,DISTURBING THE PEACE,E18,495,NaN,2018-10-03 20:13:00,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186,"(42.26260773, -71.12118637)"
1,I182080053,3201,Property Lost,PROPERTY - LOST,D14,795,NaN,2018-08-30 20:00:00,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311,"(42.35211146, -71.13531147)"
2,I182080052,2647,Other,THREATS TO DO BODILY HARM,B2,329,NaN,2018-10-03 19:20:00,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930,"(42.30812619, -71.07692974)"
3,I182080051,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,92,NaN,2018-10-03 20:00:00,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648,"(42.35945371, -71.05964817)"
4,I182080050,3122,Aircraft,AIRCRAFT INCIDENTS,A7,36,NaN,2018-10-03 20:49:00,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663,"(42.37525782, -71.02466343)"
5,I182080049,1402,Vandalism,VANDALISM,C11,351,NaN,2018-10-02 20:40:00,2018,10,Tuesday,20,Part Two,DORCHESTER AVE,42.299197,-71.060470,"(42.29919694, -71.06046974)"
6,I182080048,3803,Motor Vehicle Accident Response,M/V ACCIDENT - PERSONAL INJURY,NaN,,NaN,2018-10-03 20:16:00,2018,10,Wednesday,20,Part Three,NaN,42.320734,-71.056764,"(42.32073413, -71.05676415)"
7,I182080047,3301,Verbal Disputes,VERBAL DISPUTE,B2,603,NaN,2018-10-03 19:32:00,2018,10,Wednesday,19,Part Three,TREMONT ST,42.333807,-71.103778,"(42.33380683, -71.10377843)"
8,I182080045,802,Simple Assault,ASSAULT SIMPLE - BATTERY,E18,543,NaN,2018-10-03 19:27:51,2018,10,Wednesday,19,Part Two,AVILA RD,42.256145,-71.128025,"(42.25614494, -71.12802506)"
9,I182080044,3410,Towed,TOWED MOTOR VEHICLE,D4,621,NaN,2018-10-03 20:00:00,2018,10,Wednesday,20,Part Three,COMMONWEALTH AVE,42.348866,-71.089363,"(42.34886600, -71.08936284)"


## __Do some data preparation__

In [3]:
# drop the columns that we don't need
processedDataFrame = initialDataFrame.drop(['Location'], axis=1)

# replace some NaN values
processedDataFrame = processedDataFrame.fillna({'SHOOTING':"Ν"})

processedDataFrame['SHOOTING'].unique()  # printToBeRemoved

array(['Ν', 'Y'], dtype=object)

In [4]:
processedDataFrame  # printToBeRemoved

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long
0,I182080058,2403,Disorderly Conduct,DISTURBING THE PEACE,E18,495,Ν,2018-10-03 20:13:00,2018,10,Wednesday,20,Part Two,ARLINGTON ST,42.262608,-71.121186
1,I182080053,3201,Property Lost,PROPERTY - LOST,D14,795,Ν,2018-08-30 20:00:00,2018,8,Thursday,20,Part Three,ALLSTON ST,42.352111,-71.135311
2,I182080052,2647,Other,THREATS TO DO BODILY HARM,B2,329,Ν,2018-10-03 19:20:00,2018,10,Wednesday,19,Part Two,DEVON ST,42.308126,-71.076930
3,I182080051,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,92,Ν,2018-10-03 20:00:00,2018,10,Wednesday,20,Part One,CAMBRIDGE ST,42.359454,-71.059648
4,I182080050,3122,Aircraft,AIRCRAFT INCIDENTS,A7,36,Ν,2018-10-03 20:49:00,2018,10,Wednesday,20,Part Three,PRESCOTT ST,42.375258,-71.024663
5,I182080049,1402,Vandalism,VANDALISM,C11,351,Ν,2018-10-02 20:40:00,2018,10,Tuesday,20,Part Two,DORCHESTER AVE,42.299197,-71.060470
6,I182080048,3803,Motor Vehicle Accident Response,M/V ACCIDENT - PERSONAL INJURY,NaN,,Ν,2018-10-03 20:16:00,2018,10,Wednesday,20,Part Three,NaN,42.320734,-71.056764
7,I182080047,3301,Verbal Disputes,VERBAL DISPUTE,B2,603,Ν,2018-10-03 19:32:00,2018,10,Wednesday,19,Part Three,TREMONT ST,42.333807,-71.103778
8,I182080045,802,Simple Assault,ASSAULT SIMPLE - BATTERY,E18,543,Ν,2018-10-03 19:27:51,2018,10,Wednesday,19,Part Two,AVILA RD,42.256145,-71.128025
9,I182080044,3410,Towed,TOWED MOTOR VEHICLE,D4,621,Ν,2018-10-03 20:00:00,2018,10,Wednesday,20,Part Three,COMMONWEALTH AVE,42.348866,-71.089363


## __Data Research__

1. #### Multitude of crimes by year, by month, by date and by district

In [5]:
# groupBy year
yearMultitudeDf = processedDataFrame.groupby(['YEAR']).count()

# replace month numbers to month names
monthMultitudeDf = processedDataFrame.copy()
monthMultitudeDf['MONTH'] = monthMultitudeDf['MONTH'].apply(lambda x: calendar.month_abbr[x])

# groupBy month
monthMultitudeDf = monthMultitudeDf.groupby(['MONTH']).count()

# groupBy day
dayMultitudeDf = processedDataFrame.groupby(['DAY_OF_WEEK']).count()

# groupBy district
districtMultitudeDf = processedDataFrame.groupby(['DISTRICT']).count()

print("Count by year:")  # printToBeRemoved
print(yearMultitudeDf['INCIDENT_NUMBER'])  # printToBeRemoved
print("--------------")  # printToBeRemoved

print("Count by month:")  # printToBeRemoved
print(monthMultitudeDf['INCIDENT_NUMBER'])  # printToBeRemoved
print("--------------")  # printToBeRemoved

print("Count by day:")  # printToBeRemoved
print(dayMultitudeDf['INCIDENT_NUMBER'])  # printToBeRemoved
print("--------------")  # printToBeRemoved

print("Count by district:")  # printToBeRemoved
print(districtMultitudeDf['INCIDENT_NUMBER'])  # printToBeRemoved
print("--------------")  # printToBeRemoved

Count by year:
YEAR
2015     53392
2016     99134
2017    100938
2018     74356
Name: INCIDENT_NUMBER, dtype: int64
--------------
Count by month:
MONTH
Apr    24108
Aug    35137
Dec    23484
Feb    21661
Jan    23625
Jul    34640
Jun    30622
Mar    24156
May    26242
Nov    23685
Oct    26437
Sep    34023
Name: INCIDENT_NUMBER, dtype: int64
--------------
Count by day:
DAY_OF_WEEK
Friday       49758
Monday       46970
Saturday     45969
Sunday       41374
Thursday     47872
Tuesday      47726
Wednesday    48151
Name: INCIDENT_NUMBER, dtype: int64
--------------
Count by district:
DISTRICT
A1     36735
A15     6663
A7     13634
B2     51288
B3     36400
C11    43817
C6     24190
D14    20632
D4     43338
E13    17981
E18    17825
E5     13543
Name: INCIDENT_NUMBER, dtype: int64
--------------
